# Object detection with in images using cv2 and tf2.0

In [54]:
from pathlib import Path
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.python import ops
import sys
import time
import psutil

In [29]:
print(tf.__version__)

2.2.0


In [30]:
%matplotlib inline

In [31]:
def create_input_tensor(image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]
    return input_tensor

In [32]:
image = Image.open('./images/Keller-Sutter.jpg')

In [33]:
input_tensor = create_input_tensor(image)

In [61]:
#model_name = "ssd_inception_v2_coco_2018_01_28"
#model_name = "ssd_mobilenet_v2_coco_2018_03_29"
model_name = "faster_rcnn_inception_v2_coco_2018_01_28"

In [62]:
def download_model(model_name):
    """Loading the model from the url"""
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
      fname=model_name,
      origin=base_url + model_file,
      untar=True,
      cache_dir = "models") # save to file directory
    return model_dir

In [63]:
model_dir = download_model(model_name)

149127168/149119618 [==============================] - 118s 1us/step


In [64]:
model_dir = Path.cwd().joinpath('models', 'datasets', model_name, 'saved_model') 
print(model_dir)

G:\Udacity Edge AI ND\Project 1\models\datasets\faster_rcnn_inception_v2_coco_2018_01_28\saved_model


In [65]:
model = tf.saved_model.load(str(model_dir), tags=None)
model = model.signatures['serving_default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [66]:
total_bytes = psutil.virtual_memory()[3]
MB = total_bytes/(1024*1024)
print(MB)

10625.02734375


In [67]:
number_inferences = 100
# run inference for multiple times
start_time = time.time()
for count in range(number_inferences):
    if count % 10==0:
        print(count)
    # get prediction
    output_dict = model(input_tensor)
duration = time.time() - start_time
avg_duration = duration /number_inferences
print(f"Model name: {model_name}")
print(f"Average inference time: {avg_duration}s")

0
10
20
30
40
50
60
70
80
90
Model name: faster_rcnn_inception_v2_coco_2018_01_28
Average inference time: 0.6033855366706848s


In [50]:
num_detections = int(output_dict.pop('num_detections'))
output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
output_dict['num_detections'] = num_detections

# detection_classes should be ints.
output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

In [53]:
boxes = output_dict['detection_boxes']
print(boxes)

[[0.10947913 0.22465327 1.         0.7916198 ]]
